In [128]:
import sqlite3
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

IMPORT DATA

In [129]:
conn = sqlite3.connect('database.sqlite')

player_attribute = pd.read_sql('select * from player_attributes ',conn)
country = pd.read_sql('select * from country', conn)
league = pd.read_sql('select * from league',conn)
match = pd.read_sql('select * from match', conn)
player = pd.read_sql('select * from player',conn)
plater_attribute = pd.read_sql('select * from player_attributes',conn)
team_attributes = pd.read_sql('select * from team_attributes',conn)
team = pd.read_sql('select * from team',conn)

In [130]:
plater_attribute.head(2)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


In [131]:
#Merge Country and League data
countries_league = country.merge(league,left_on='id', right_on='id', how='outer')
countries_league.head()

,id,name_x,country_id,name_y
0,1,Belgium,1,Belgium Jupiler League
1,1729,England,1729,England Premier League
2,4769,France,4769,France Ligue 1
3,7809,Germany,7809,Germany 1. Bundesliga
4,10257,Italy,10257,Italy Serie A


In [132]:
#Drop Id column in dataframe
countries_league = countries_league.drop(['id'], axis=1)
countries_league.head()

,name_x,country_id,name_y
0,Belgium,1,Belgium Jupiler League
1,England,1729,England Premier League
2,France,4769,France Ligue 1
3,Germany,7809,Germany 1. Bundesliga
4,Italy,10257,Italy Serie A


In [133]:
#Rename column in dataframe
countries_league = countries_league.rename(columns={'Country_league':'league'} )
countries_league.head(2)

,name_x,country_id,name_y
0,Belgium,1,Belgium Jupiler League
1,England,1729,England Premier League


In [134]:
#create new dataframe by taking necessary columns 
match_new = match[['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id',
                   'away_team_api_id','home_team_goal','away_team_goal']]

In [135]:
match_new.head(2)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0


In [137]:
#Remove ID column from matchnew dataframe
match_new = match_new.drop(['id'], axis=1)

KeyError: "['id'] not found in axis"

In [138]:
match_new.head(2)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
1,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0


In [139]:
#Merge 
data = match_new.merge(countries_league, left_on='country_id', right_on='country_id', how='outer')
data.head(2)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,name_x,name_y
0,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,Belgium,Belgium Jupiler League
1,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,Belgium,Belgium Jupiler League


--------------------------------------

------------------------------------

In [140]:
player_attribute.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


In [141]:
df = player_attribute.copy()

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [143]:
df.shape

(183978, 42)

In [144]:
df.dtypes[df.dtypes==object]


date                   object
preferred_foot         object
attacking_work_rate    object
defensive_work_rate    object
dtype: object

In [145]:
#categorical feature
df[['date','preferred_foot','attacking_work_rate','defensive_work_rate']]

,date,preferred_foot,attacking_work_rate,defensive_work_rate
0,2016-02-18 00:00:00,right,medium,medium
1,2015-11-19 00:00:00,right,medium,medium
2,2015-09-21 00:00:00,right,medium,medium
3,2015-03-20 00:00:00,right,medium,medium
4,2007-02-22 00:00:00,right,medium,medium
...,...,...,...,...
183973,2009-08-30 00:00:00,right,medium,low
183974,2009-02-22 00:00:00,right,medium,low
183975,2008-08-30 00:00:00,right,medium,low
183976,2007-08-30 00:00:00,right,medium,low


In [146]:
print(df['preferred_foot'].unique())
print(df[df['preferred_foot']== 'left'].shape)
print(df[df['preferred_foot']== 'right'].shape)
print(df[df['preferred_foot']==  None].shape)

['right' 'left' None]
(44733, 42)
(138409, 42)
(0, 42)


In [147]:
#only near about 760 values are none in prefered foot so we will replace None with right

In [148]:
df['preferred_foot'] = df['preferred_foot'].map({'right':'0', 'left':'1'})
print(df['preferred_foot'].unique())

['0' '1' nan]


In [149]:
df['preferred_foot'].fillna(value=0, inplace=True)

In [150]:
#We fill up with NA value
df['preferred_foot'].isna().sum()

0

In [151]:
print(df['attacking_work_rate'].unique())
print(df[df['attacking_work_rate']== 'medium'].shape)
print(df[df['attacking_work_rate']== 'high'].shape)
print(df[df['attacking_work_rate']== 'low'].shape)
print(df[df['attacking_work_rate']== 'le'].shape)
print(df[df['attacking_work_rate']== 'norm'].shape)
print(df[df['attacking_work_rate']== 'stoc'].shape)
print(df[df['attacking_work_rate']== 'y'].shape)
print(df[df['attacking_work_rate']== None].shape)

['medium' 'high' None 'low' 'None' 'le' 'norm' 'stoc' 'y']
(125070, 42)
(42823, 42)
(8569, 42)
(104, 42)
(348, 42)
(89, 42)
(106, 42)
(0, 42)


In [152]:
#Medium value has more count
from sklearn import preprocessing
label_encoder =  preprocessing.LabelEncoder()
df['attacking_work_rate'] = label_encoder.fit_transform(df['attacking_work_rate'].astype(str))

In [153]:

df['attacking_work_rate'].unique()[:]

array([4, 1, 0, 3, 2, 5, 6, 7])

In [154]:
df['attacking_work_rate'].isna().sum()

0

In [155]:
print(df['defensive_work_rate'].unique())
print(df[df['defensive_work_rate']== 'medium'].shape)
print(df[df['defensive_work_rate']== 'high'].shape)
print(df[df['defensive_work_rate']== 'low'].shape)
print(df[df['defensive_work_rate']== '_0'].shape)
print(df[df['defensive_work_rate']== 'ean'].shape)
print(df[df['defensive_work_rate']== 'ormal'].shape)
print(df[df['defensive_work_rate']== 'es'].shape)
print(df[df['defensive_work_rate']== None].shape)

['medium' 'high' 'low' '_0' None '5' 'ean' 'o' '1' 'ormal' '7' '2' '8' '4'
 'tocky' '0' '3' '6' '9' 'es']
(130846, 42)
(27041, 42)
(18432, 42)
(2394, 42)
(104, 42)
(348, 42)
(106, 42)
(0, 42)


In [156]:
#Medium value has more count
from sklearn import preprocessing
label_encoder =  preprocessing.LabelEncoder()
df['defensive_work_rate'] = label_encoder.fit_transform(df['defensive_work_rate'].astype(str))

In [157]:
print(df['defensive_work_rate'].unique())

[16 14 15 11 10  5 12 17  1 18  7  2  8  4 19  0  3  6  9 13]


In [158]:
#We have handle all categorical data

In [159]:
df.dtypes[df.dtypes == object]

date              object
preferred_foot    object
dtype: object

Replace Nan value in complete dataset

In [160]:
for column in df.columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [161]:
df['balance'].mode()[0]

70.0

In [162]:
df.isna().any().sum()

0

till here we have handle categorical and Nan data

In [163]:
df.head(2)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,0,4,16,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,0,4,16,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


In [164]:
df.drop(['player_fifa_api_id','player_api_id','date'], axis=1, inplace=True)

In [165]:
df.head(2)

,id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,67.0,71.0,0,4,16,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,67.0,71.0,0,4,16,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


In [166]:
y = df['overall_rating']
x= df.drop('overall_rating', axis=1)

In [167]:
print(x.shape)
print(y.shape)

(183978, 38)
(183978,)


In [168]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


In [169]:

from statsmodels.stats.outliers_influence import variance_inflation_factor 

vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif["Features"] = x.columns

#let's check the values
vif

,vif,Features
0,1.003059,id
1,2.140865,potential
2,1.079537,preferred_foot
3,1.252305,attacking_work_rate
4,1.160021,defensive_work_rate
5,4.753290,crossing
6,6.949150,finishing
7,4.168554,heading_accuracy
8,8.767753,short_passing
9,5.036370,volleys


In [170]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.30, random_state=855)

LINEAR REGRESSION

In [171]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [172]:
y_pred = lr.predict(x_test)
lr.score(x_test,y_test)

0.8414081784499042

DECISION TREE

In [173]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth =24, min_samples_leaf= 1, min_samples_split= 2, splitter ='random')
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.652208573395659

In [174]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [175]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(x_train, y_train)
knn.score(x_test, y_test)

0.8753665435884446

In [176]:
y_pred = knn.predict(x_test)

array([69.8, 69.2, 68. , ..., 70. , 64.2, 61.2])

In [126]:
rand_reg = RandomForestRegressor(n_estimators=5 )
rand_reg.fit(x_train,y_train)
rand_reg.score(x_test,y_test)

0.973666495189718

In [194]:
Pred = pd.DataFrame({'Actual Points': y_test.tolist(), 'Predicted Points': y_pred.tolist()})


In [195]:
Pred.head(50)

,Actual Points,Predicted Points
0,69.0,69.8
1,68.0,69.2
2,69.0,68.0
3,69.0,69.2
4,70.0,71.8
5,73.0,72.4
6,61.0,64.8
7,59.0,62.0
8,63.0,60.6
9,60.0,60.4
